In [54]:
import os
import json
import cv2

In [58]:
# video = "IKF_8.27_230pm_BW_Front_P11.webm" # Good
video = "IKF_8.27_230pm_BW_Front2_P10.webm" # Bad
keypoints_directory = "../../data/keypoints"
videos_directory = "../../data/source_videos"
# keypoints_data_dir = f"{keypoints_directory}/run-preproccessed-1697924524790004000/IKF_8.27_230pm_BW_Front_P11-1697925384052075000" # Good
keypoints_data_dir = f"{keypoints_directory}/run-preproccessed-1697924524790004000/IKF_8.27_230pm_BW_Front2_P10-1697925482541409000" # Bad
print(keypoints_data_dir)
keypoints_video = f"{videos_directory}/{video}"

../../data/keypoints/run-preproccessed-1697924524790004000/IKF_8.27_230pm_BW_Front2_P10-1697925482541409000


In [59]:
video_keypoint_data = []
for f in sorted(os.listdir(keypoints_data_dir)):
    with open(f"{keypoints_data_dir}/{f}", 'r') as keypoints_json:
       video_keypoint_data.append(json.load(keypoints_json))

In [60]:
keypoints = [k for k in video_keypoint_data[0]['joint_positions'].keys()]
print(keypoints)

['nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner', 'right_eye', 'right_eye_outer', 'left_ear', 'right_ear', 'mouth_left', 'mouth_right', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky', 'left_index', 'right_index', 'left_thumb', 'right_thumb', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel', 'right_heel', 'left_foot_index', 'right_foot_index']


In [63]:
keypoints = [k for k in video_keypoint_data[0]['joint_positions'].keys()]
cap = cv2.VideoCapture(f"{videos_directory}/{video}")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_size = (frame_width,frame_height)
fps = 20

cv2.startWindowThread()
frame = 0
skel = [
    ('nose', 'left_eye'),
    ('nose', 'right_eye'),
    ('nose', 'left_shoulder'),
    ('nose', 'right_shoulder'),
    ('left_shoulder', 'right_shoulder'),
    ('left_shoulder', 'left_elbow'),
    ('left_shoulder', 'left_hip'),
    ('left_elbow', 'left_wrist'),
    ('right_elbow', 'right_wrist'),
    ['left_wrist', 'left_pinky'],
    ['right_wrist', 'right_pinky'],
    ('right_shoulder', 'right_elbow'),
    ('right_shoulder', 'right_hip'),
    ('right_hip', 'left_hip'),
    ('right_hip', 'right_knee'),
    ('left_hip', 'left_knee'),
    ('left_heel', 'left_knee'),
    ('left_heel', 'left_foot_index'),
    ('right_heel', 'right_knee'),
    ('right_heel', 'right_foot_index'),
]
output = cv2.VideoWriter('../../data/sample.mov',  
                         cv2.VideoWriter_fourcc(*'mp4v'), 
                         10, (1080, 720)) 
while cap.isOpened():
    ret, image = cap.read()
    h, w, _ = image.shape

    keypoint_data = video_keypoint_data[frame]
    for k in keypoints:
        keypoint_data = video_keypoint_data[frame]['joint_positions'][k]
        for s in skel:
            a = video_keypoint_data[frame]['joint_positions'][s[0]]
            b = video_keypoint_data[frame]['joint_positions'][s[1]]
            xa = int(a['x'] * w)
            xb = int(b['x'] * w)
            ya = int(a['y'] * h)
            yb = int(b['y'] * h)
            cv2.line(image,(xa, ya), (xb,yb), (0 ,255 ,0), 2)


    output.write(image)
    cv2.imshow('frame', image) 
    cv2.waitKey(1)
    
    frame += 1
    if not ret or frame >= len(video_keypoint_data):
        output.release()
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break

